In [2]:
import numpy as np
from data_prep import features, targets, features_test, targets_test

np.random.seed(21)

def sigmoid(x):
    """
    Calculate the sigmoid activation function.
    """
    return 1 / (1 + np.exp(-np.array(x, dtype=float)))

# Hyperparameters
n_hidden = 2  # Number of hidden units
epochs = 900  # Number of training iterations
learnrate = 0.005  # Learning rate
n_records, n_features = features.shape  # Number of records and features
last_loss = None  # Track the loss trend

# Initialize weights with normal distribution scaled by 1/sqrt(n_features)
weights_input_hidden = np.random.normal(scale=1/n_features**0.5, size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1/n_features**0.5, size=n_hidden)

# Training loop
for e in range(epochs):
    # Initialize weight changes
    del_w_input_hidden = np.zeros(weights_input_hidden.shape, dtype=float)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape, dtype=float)
    
    for x, y in zip(features.values, targets):
        x = x.astype(float)  # Ensure x is float to prevent dtype errors

        ## Forward pass ##
        hidden_input = np.dot(x, weights_input_hidden)  # Compute input to hidden layer
        hidden_output = sigmoid(hidden_input)  # Apply activation function
        
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))  # Output layer
        
        ## Backward pass ##
        error = y - output  # Compute error
        
        output_error_term = error * output * (1 - output)  # Compute error term for output layer
        
        hidden_error = weights_hidden_output * output_error_term  # Backpropagate error to hidden layer
        
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)  # Compute hidden layer error term
        
        # Update the weight changes (gradient accumulation)
        del_w_hidden_output += output_error_term * hidden_output  
        del_w_input_hidden += hidden_error_term * x[:, None]  

    # Update weights (divide by n_records for batch gradient descent)
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records
    
    # Monitor training loss
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(features, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output, weights_hidden_output))
        loss = np.mean((out - targets) ** 2)  # Compute Mean Squared Error
        if last_loss and last_loss < loss:
            print(f"Train loss: {loss:.6f}  WARNING - Loss Increasing")
        else:
            print(f"Train loss: {loss:.6f}")
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)

print(f"Prediction accuracy: {accuracy:.3f}")

### **Notebook grading**
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
test_accuracy = np.mean(predictions == targets_test)

if test_accuracy and test_accuracy > 0.7:
    print("Nice job! That's right!")
else:
    print("Your accuracy is too low.")



Train loss: 0.265901
Train loss: 0.264289
Train loss: 0.262721
Train loss: 0.261195
Train loss: 0.259710
Train loss: 0.258266
Train loss: 0.256862
Train loss: 0.255497
Train loss: 0.254170
Train loss: 0.252880
Prediction accuracy: 0.425
Your accuracy is too low.
